In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import json

In [80]:
api_keys = json.load(open('api_keys.json', 'r'))

# 1. 주소 있는 데이터 읽어서 확인 

In [14]:
df = pd.read_csv('./jonmat2017.csv')
df = df[df.columns[:-1]]

In [15]:
df = df[~df.주소.isnull()]
df[:3]

,총연번,의원번호,의원명,당,당ID,지역명,연월일,내역,지출액,사용처,분류,주소,의원지출액순위
345,17_001545,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.1.11,지역현안관련정책개발보좌직원간담회,"105,000",또래오래치킨,사무실_식대비,서울특별시 영등포구 여의대방로68길 17,4
346,17_001554,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.1.17,상임위법안소위관련보좌직원정책개발간담회,"160,000",청도,사무실_식대비,서울특별시 영등포구 여의도동 45-20 동북빌딩 203-204,4
347,17_001556,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.1.17,정책개발기자간담회,"350,000",잔비어스,언론_기자식대등,서울특별시 강남구 개포로 623,4


In [16]:
df['주소'] = df['주소'].apply(lambda x: x.strip() if type(x)==str else None)
df['지출액'] = df['지출액'].apply(lambda x: int(x.replace(',','')))

# 2. 주소 있는것 지오코딩

In [19]:
import googlemaps
from datetime import datetime

gmaps = googlemaps.Client(key=api_keys['google_maps'])

In [20]:
def get_geocode(addr):
    geocode_result = gmaps.geocode(addr)
    if len(geocode_result) > 0:
        return geocode_result[0]['geometry']['location']
    else:
        return {'lat':0,'lng':0}

In [38]:
uniq = df['주소'].unique()

In [43]:
try:
    addr_dict = json.load(open('geocode_dict.json', 'r'))
except:
    addr_dict = {}

In [44]:
for addr in uniq:
    if addr in addr_dict:
        continue
    addr_dict[addr] = get_geocode(addr)

In [45]:
json.dump(addr_dict, open('geocode_dict.json', 'w'))

In [46]:
for enum in enumerate(addr_dict):
    if enum[0] > 2:
        break
    print(enum)

(0, '서울특별시 영등포구 여의대방로68길 17')
(1, '서울특별시 영등포구 여의도동 45-20 동북빌딩 203-204')
(2, '서울특별시 강남구 개포로 623')


In [49]:
df_latlng = df['주소'].apply(lambda x: pd.Series({f:addr_dict[x][f] for f in ['lat','lng']}))
df_latlng[:3]

,lat,lng
345,37.517729,126.932872
346,37.519623,126.928920
347,37.493935,127.079623


In [54]:
df = pd.concat([df, df_latlng], axis=1)

In [56]:
df[:3].T

,345,346,347
총연번,17_001545,17_001554,17_001556
의원번호,5,5,5
의원명,강석호,강석호,강석호
당,자유한국당,자유한국당,자유한국당
당ID,200,200,200
지역명,경북 영양군영덕군봉화군울진군,경북 영양군영덕군봉화군울진군,경북 영양군영덕군봉화군울진군
연월일,2017.1.11,2017.1.17,2017.1.17
내역,지역현안관련정책개발보좌직원간담회,상임위법안소위관련보좌직원정책개발간담회,정책개발기자간담회
지출액,105000,160000,350000
사용처,또래오래치킨,청도,잔비어스


# 3. 같은좌표의 다른 주소 표현
(도로명, 건물명, 띄어쓰기 등)

In [57]:
def uniq_cnt(l):
    return len(l.unique())

In [83]:
addr_grouped = df.groupby(['사용처','lat','lng'])\
                .agg({'주소':[lambda x: len(x.unique()), min]})
addr_grouped[:3]

주소                      
                                  <lambda>                   min
사용처          lat       lng                                      
(주)거구상사      37.552459 126.937826        1      서울특별시 마포구 백범로 23
(주)미가케이터링컴퍼니 37.519749 126.929719        1    서울특별시 영등포구 의사당대로 1
(주)밥보        37.530234 126.921678        1  서욽특별시 영등포구 여의도동 13-3

In [84]:
addr_grouped.columns = ['uniq_cnt','addr']

In [104]:
addr_grouped = addr_grouped.query("uniq_cnt > 1").reset_index()
addr_grouped[:3]

,사용처,lat,lng,uniq_cnt,addr
0,(주)이랜드파크켄싱턴여의도,37.530234,126.921678,2,서울특별시 영등포구 국회대로76길 16
1,(주)커피빈코리아,37.437736,126.786437,2,경기도 시흥시 산천로 100번안길 16
2,고래와새우,37.529955,126.919895,3,서울특별시 영등포구 국회대로 70길 7


In [119]:
def change_addr(series_place):
    print(df_all[df_all.사용처==series_place['사용처']])

In [120]:
addr_grouped[:3].apply(change_addr, axis=1)

             총연번  의원번호     의원명       당   당ID              지역명         연월일  \
404    17_001836     5     강석호   자유한국당   200  경북 영양군영덕군봉화군울진군   2017.6.15   
2659   17_017916   355  김선동(서)   자유한국당   200          서울 도봉구을   2017.3.31   
3773   17_022850    44     김영주  더불어민주당   100         서울 영등포구갑    2017.4.6   
3937   17_024203    48     김용태   자유한국당   200          서울 양천구을    2017.3.5   
4709   17_029024   372     김종회    국민의당  2050        전북 김제시부안군   2017.8.31   
4736   17_029148   372     김종회    국민의당  2050        전북 김제시부안군  2017.10.18   
6457   17_041186    97     민병두  더불어민주당   100         서울 동대문구을   2017.10.9   
8658   17_057385   140     서청원   자유한국당   200          경기 화성시갑   2017.3.31   
8688   17_057541   140     서청원   자유한국당   200          경기 화성시갑    2017.9.6   
9521   17_062609   406     송영길  더불어민주당   100          인천 계양구을   2017.1.25   
9524   17_062623   406     송영길  더불어민주당   100          인천 계양구을    2017.2.7   
9550   17_062735   406     송영길  더불어민주당   100          인천 계양구을   2017.4.13   

0    None
1    None
2    None
dtype: object

# 기타(정리 안 함)

In [40]:
import requests
import json

In [90]:
df_all = pd.read_csv('jonmat2017.csv')
place_dict = {}

In [75]:
import requests
import json
import time

class NaverMaps():
    headers = {}
    url_format = "https://naveropenapi.apigw.ntruss.com/map-place/v1/"+\
        "search?query=%s&coordinate=126.929719,37.519749"
    
    def __init__(self, api_key_id, api_key):
        self.headers = {"X-NCP-APIGW-API-KEY-ID":api_key_id,
                        "X-NCP-APIGW-API-KEY":api_key,
                    }
    
    def search(self, query_string, retry=5):
        query_string = query_string.replace('#','')
        r = requests.get(self.url_format % query_string, headers=self.headers)
        d = json.loads(r.text)
        if d['status'] != 'OK':
            if retry > 0:
                print("%s retry: %s" % (place_name, retry))
                time.sleep(3)
                return self.search(query_string, retry=retry-1)
            else:
                raise BaseException(query_string, d)
        return d

In [77]:
m = NaverMaps(api_keys['naver_maps_id'], api_keys['naver_maps_key'])
r = m.search('비야게레로')
r

{'status': 'OK',
 'meta': {'totalCount': 1, 'count': 1},
 'places': [{'name': '비야게레로',
   'road_address': '서울특별시 강남구 봉은사로78길 12',
   'jibun_address': '서울특별시 강남구 삼성동 118-21 101호',
   'phone_number': '02-538-8915',
   'x': '127.0528107',
   'y': '37.5119717',
   'distance': 10883.506983308978,
   'sessionId': 'C8AqpmkBLhmWpuc1KvlZ'}],
 'errorMessage': ''}

In [119]:
for place_name in df['사용처'].unique():
    if place_name not in place_dict:
        #print("requested")
        r = m.search(place_name)
        place_dict[place_name] = r['places']
    #print(place_name, len(place_dict[place_name]))

(주)파리크라상천등산(평택방향) retry: 5
(주)파리크라상천등산(평택방향) retry: 4
친구찾기 retry: 5
친구찾기 retry: 4
우동전문점 retry: 5
신안홍탁 retry: 5
(주)부일씨앤디 retry: 5
옛삼덕수타면 retry: 5
가매야 retry: 5
차아나플레인 retry: 5
장수갈매기 retry: 5
해미집 retry: 5
올라6(주)호텔올라 retry: 5
올라6(주)호텔올라 retry: 4
주식회사공바위 retry: 5
고려정 retry: 5
한서외식산업주식회사(한우리한점) retry: 5
창고43시청점 retry: 5
광안리부산환집 retry: 5
0(1조 retry: 5
리틀타이 retry: 5
스타벅스(서여의도) retry: 5
(주)블랙스톤리조트양평 retry: 5
한화호텔앤드리조트(의원식당) retry: 5
행복의강 retry: 5
신씨화로신촌점 retry: 5
오늘은재영이네가 retry: 5
국회후생복지(커피숍) retry: 5
백리향(63빌딩) retry: 5
김박사손칼국수 retry: 5
돌산산장 retry: 5
삼송 retry: 5
금호횟집 retry: 5
호텔농심 retry: 5
호메로스호텔 retry: 5
해주냉면 retry: 5
주례한우갈비 retry: 5
어부횟집 retry: 5
향촌돼지국밥 retry: 5
여울돈단지 retry: 5
홍문 retry: 5
리안유통 retry: 5
구통성 retry: 5
노이프레소커피 retry: 5
마산식당 retry: 5
마산식당 retry: 4
한상바다 retry: 5
농업회사법인(주)대성축산물직판장 retry: 5
양평(하)휴게소탐앤탐스 retry: 5
땀뽕 retry: 5
제일면제소 retry: 5
자성화코다리 retry: 5
진사골순대국설렁탕 retry: 5
세프의국수전 retry: 5
월향조선횟집 retry: 5
한길식당 retry: 5
정읍(논산)휴게소 retry: 5
용인(강릉)휴게소 retry: 5
주식회사설풍 retry: 5
바다호텔 retry: 

In [131]:
json.dump(place_dict, open('./place_dict.json', 'w'))

In [120]:
len(place_dict)

6762

In [125]:
pd.Series([len(place_dict[place]) for place in place_dict]).value_counts()

5    2288
0    2201
1    1367
2     470
3     247
4     189
dtype: int64

In [134]:
pd.DataFrame(data={'road_address': '서울특별시 영등포구 국회대로 800 진미파라곤',
  'phone_number': '02-780-0363',
  'x': '126.9213120',
  'y': '37.5309780'}, index=[1])

,road_address,phone_number,x,y
1,서울특별시 영등포구 국회대로 800 진미파라곤,02-780-0363,126.9213120,37.5309780


In [198]:
def place_info(place_name, elem):
    d = place_dict[place_name]
    if len(d) != 1:
        return None
    if elem=='road_address' and d[0][elem]=='':
        elem='jibun_address'
    return d[0][elem]

In [199]:
x = []
columns = ['road_address','phone_number','x','y']
for c in columns:
    x.append(df['사용처'].apply(place_info, elem=c))

In [243]:
def place_series(place_name, return_all=False):
    d = place_dict[place_name]
    if len(d) == 0 or \
    ((not return_all) and len(d) > 1): # return that has only one match
        return pd.Series()
    
    addr_column = 'road_address'
    if d[0][addr_column]=='': # some places have no "road address"
        addr_column='jibun_address'
    return_dict = {elem:d[0][elem] for elem in ['phone_number','x','y']}
    return_dict['addr'] = d[0][addr_column]
    return pd.Series(return_dict)

In [245]:
naver_map_df = df['사용처'].apply(place_series)

In [258]:
df = pd.concat([df, naver_map_df], axis=1)

In [294]:
addr_minmax = df[df.notnull()['주소']].groupby(['의원명','사용처']).agg({'주소':[min, max]})
addr_minmax[:5]

주소                           
                                          min                        max
의원명 사용처                                                                 
강석호 (주)밥보                서욽특별시 영등포구 여의도동 13-3       서욽특별시 영등포구 여의도동 13-3
    (주)이랜드파크켄싱턴여의도       서울특별시 영등포구 여의도동 13-3       서울특별시 영등포구 여의도동 13-3
    (주)이와타쇼         서울특별시 영등포구 여의도동 36-2 지하1층  서울특별시 영등포구 여의도동 36-2 지하1층
    (주)제스코푸드청담지점             서울특별시 강남구 삼성동 88           서울특별시 강남구 삼성동 88
    (주)조선호텔호무랑              서울특별시 강남구 청담동 4-1          서울특별시 강남구 청담동 4-1

In [295]:
tmp_df = addr_minmax.droplevel(level=0, axis=1)

In [296]:
tmp_df[tmp_df['min'] != tmp_df['max']]

min  \
의원명 사용처                                               
김성수 (주)커피빈코리아                 경기도 시흥시 산천로 100번안길 16   
    물자리                         서울특별시 종로구 삼봉로 80-10   
    봉명성                            서울특별시 영등포구 74길 9   
    스시설                   경기도 고양시 일산동구 강송로 217번길 73   
    이꾸                        서울특별시 영등포구 국회대로 72길 6   
    정인면옥                     서울특별시 영등포구 국회대로 76길 22   
    진진                  서울특별시 영등포구 국회대로 72길 11 프린스텔   
    차이나플레인                   서울특별시 영등포구 국회대로 72길 11   
    쿠치나후                         서울특별시 영등포구 의사당대로 1   
    파리크라상                         서울특별시 영등포구 74길 12   
    파리크루아상                        서울특별시 영등포구 74길 12   
노웅래 군산오징어                      서울특별시 강남구 남부순환로 2615   
    양평신내해장국                    서울특별시 마포구 토정로 31길 24   
    지호마포점                         서울특별시 마포구 독막로 229   
    회관직원식당                      서울특별시 마포구 마포대로6길 10   
서청원 상도갈비타운                          서울시 동작구 상도동 522   
유승민 삼보정                        서울특별시 영등포구 여의도동 35-5   
    한화호텔앤리조트프레스센터점                서울특별시 중구 세종대로 124   
    향연                         서울특별시 영등포구 여의도동 23-1   
정우택 노조미             서울특별시 영등포구 여의도동 15-24 삼희익스콘벤처타워   
    차이나프로               서울특별시 영등포구 여의도동 LG트윈타워 동관5층   

                                                      max  
의원명 사용처                                                    
김성수 (주)커피빈코리아                       경기도 시흥시 신천로 100번안길 16  
    물자리                               서울특별시 종로구 상봉로 80-10  
    봉명성                             서울특별시 영등포구 국회대로 74길 9  
    스시설                          경기도 고양시 일산동구 강송로217번길 73  
    이꾸                               서울특별시 영등포구 국회대로72길 6  
    정인면옥                          서울특별시 영등포구 국회대로 76번길 22  
    진진                             서울특별시 영등포구 국회대로 76길 11  
    차이나플레인                           서울특별시 영등포구 여의도동 13-3  
    쿠치나후                            서울특별시 영등포구 의사당대로1 의정관  
    파리크라상                          서울특별시 영등포구 국회대로 74길 12  
    파리크루아상                         서울특별시 영등포구 국회대로 74길 12  
노웅래 군산오징어                          서울특별시 마포구 마포대로 14가길 18  
    양평신내해장국                           서울특별시 마포구 토정로31길 24  
    지호마포점                           서울특별시 영등포구 국회대로74길 19  
    회관직원식당                             서울특별시 영등포구 의사당대로 1  
서청원 상도갈비타운                              서울특별시 동작구 상도동 522  
유승민 삼보정                         서울특별시 영등포구 여의도동 35-5 종합상가  
    한화호텔앤리조트프레스센터점                       서울특별시 중구 세종대로124  
    향연                              서울특별시 종로구 사직로 10길 9-6  
정우택 노조미             서울특별시 영등포구 여의도동 15-24 삼희익스콘벤처타워 B113호  
    차이나프로                        서울특별시 영등포구 여의도동 맨하탄빌딩 1층

In [328]:
df.query("주소>'' and addr>''")

,총연번,의원번호,의원명,당,당ID,지역명,연월일,내역,지출액,사용처,분류,주소,의원지출액순위,Unnamed: 13,phone_number,x,y,addr
351,17_001581,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.2.7,지역현안관련의견청취기자간담회,"232,000",해우리서여의도점,언론_기자식대등,서울특별시 영등포구 여의도동 16-2 1층,4,NaN,02-761-4997,126.9226925,37.5281688,서울특별시 영등포구 은행로 30 중소기업중앙회
356,17_001591,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.2.13,전국위원회관련간담회,"180,600",해우리서여의도점,간담회_식대,서울특별시 영등포구 여의도동 16-2 1층,4,NaN,02-761-4997,126.9226925,37.5281688,서울특별시 영등포구 은행로 30 중소기업중앙회
360,17_001609,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.2.20,국정현안관련의견청취기자간담회,"160,000",스시나고미,언론_기자식대등,서울특별시 영등포구 국회대로 780,4,NaN,02-783-6409,126.9191480,37.5299082,서울특별시 영등포구 국회대로 780 엘지에클라트오피스텔 B105호
363,17_001619,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.2.22,정책개발기자간담회,"275,700",해우리서여의도점,언론_기자식대등,서울특별시 영등포구 여의도동 16-2 1층,4,NaN,02-761-4997,126.9226925,37.5281688,서울특별시 영등포구 은행로 30 중소기업중앙회
378,17_001680,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.3.29,지역주요현안및상임위관련등정책개발보좌직원간담회,"474,200",나리스키친,사무실_식대비,서울특별시 영등포구 여의도동 23-7,4,NaN,032-620-5690,126.7785336,37.4970566,경기도 부천시 신흥로 150
379,17_001681,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.3.29,지역주요현안및상임위관련등정책개발보좌직원간담회,"79,000",나리스키친,사무실_식대비,서울특별시 영등포구 여의도동 23-7,4,NaN,032-620-5690,126.7785336,37.4970566,경기도 부천시 신흥로 150
382,17_001692,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.4.6,정책개발기자간담회,"263,000",장어가조타,언론_기자식대등,서울특별시 영등포구 여의도동 36,4,NaN,02-780-6030,126.9264116,37.5201766,서울특별시 영등포구 의사당대로 127
384,17_001697,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.4.11,정책개발보좌직원간담회,"232,000",해우리서여의도점,사무실_식대비,서울특별시 영등포구 여의도동 16-2 1층,4,NaN,02-761-4997,126.9226925,37.5281688,서울특별시 영등포구 은행로 30 중소기업중앙회
387,17_001701,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.4.12,정책개발기자간담회,"313,410",나리스키친,언론_기자식대등,서울특별시 영등포구 여의도동 23-7,4,NaN,032-620-5690,126.7785336,37.4970566,경기도 부천시 신흥로 150
393,17_001717,5,강석호,자유한국당,200,경북 영양군영덕군봉화군울진군,2017.4.18,정책개발보좌직원간담회(진주출장),"325,000",천황식당,사무실_식대비,경상남도 진주시 대안동 4-1,4,NaN,055-741-2646,128.0844789,35.1961301,경상남도 진주시 촉석로207번길 3
